### To run the cbgt network, see first the technical_testbench to install all the required packages and start the server 

In [ ]:
#Auto reloading 
#%load_ext autoreload
#%autoreload 2
#%reload_ext autoreload 

In [ ]:
#Removing indentation issues:
#!autopep8 --in-place --aggressive --aggressive init_params.py
#!autopep8 --in-place --aggressive --aggressive popconstruct.py
#!autopep8 --in-place --aggressive --aggressive generateepochs.py
#!autopep8 --in-place --aggressive --aggressive qvalues.py

In [1]:
!python setup.py build_ext --inplace

running build_ext
copying build/lib.linux-x86_64-3.6/agent_timestep.cpython-36m-x86_64-linux-gnu.so -> 


In [2]:
#Importing scripts:
import cbgt as cbgt
from frontendhelpers import * 
from tracetype import *
import init_params as par 
import popconstruct as popconstruct
import qvalues as qval
import generateepochs as gen
from agentmatrixinit import *
#from general_init_codeblocks import *
from agent_timestep import timestep_mutator, multitimestep_mutator
import pdb

/home/bahuguna/anaconda3/envs/cbgt_env/lib/python3.6/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
2021-09-06 11:43:53,713	INFO worker.py:727 -- Connecting to existing Ray cluster at address: 10.0.0.154:6379


# Create pipelines

### Network pipeline

In [3]:
#Defining necessary codeblocks:

#MODIFIERS:

#init_params.py: to modify the neuronal default values

def codeblock_modifycelldefaults(self):
    self.celldefaults = par.helper_cellparams(self.params)

def codeblock_modifypopspecific(self):
    self.popspecific = par.helper_popspecific(self.pops)

def codeblock_modifyreceptordefaults(self):
    self.receptordefaults = par.helper_receptor(self.receps)

def codeblock_modifybasestim(self):
    self.basestim = par.helper_basestim(self.base)

def codeblock_modifydpmndefaults(self):
    self.dpmndefaults = par.helper_dpmn(self.dpmns)

def codeblock_modifyd1defaults(self):
    self.d1defaults = par.helper_d1(self.d1)

def codeblock_modifyd2defaults(self):
    self.d2defaults = par.helper_d2(self.d2)

def codeblock_modifyactionchannels(self):
    self.actionchannels = par.helper_actionchannels(self.channels)

#popconstruct.py: to modify population parameters

def codeblock_popconstruct(self):
    self.popdata = popconstruct.helper_popconstruct(self.actionchannels, self.popspecific, self.celldefaults, self.receptordefaults, self.basestim, self.dpmndefaults, self.d1defaults, self.d2defaults)

def codeblock_poppathways(self):
    self.pathways = popconstruct.helper_poppathways(self.popdata, self.newpathways)

#init_params.py: Q-values initialization and update

def codeblock_init_Q_support_params(self):
    self.Q_support_params = qval.helper_init_Q_support_params()

def codeblock_update_Q_support_params(self,reward_val, chosen_action):
    self.Q_support_params = qval.helper_update_Q_support_params(self.Q_support_params,self.reward_val,pl.chosen_action)

def codeblock_init_Q_df(self):
    self.Q_df = qval.helper_init_Q_df(self.channels)

def codeblock_update_Q_df(self):
    self.Q_df, self.Q_support_params, self.dpmndefaults = qval.helper_update_Q_df(self.Q_df, self.Q_support_params, self.dpmndefaults,pl.trial_num)


In [4]:
pl = cbgt.Pipeline()

### Reward pipeline

In [5]:
rsg = cbgt.Pipeline() #rsg is short for 'reward schedule generator'


(rsg.volatile_pattern,
 rsg.cp_idx,
 rsg.cp_indicator,
 rsg.noisy_pattern,
 rsg.t_epochs
) = rsg[gen.GenRewardSchedule](
        rsg.n_trials,
        rsg.volatility,
        rsg.conflict,
        rsg.reward_mu,
        rsg.reward_std, pl.actionchannels
    ).shape(5)

In [6]:
#Adding rsg pipeline to the network pipeline: 
pl.add(rsg) 

In [7]:
#to update the Q-values 
pl.trial_num = 0 #first row of Q-values df - initialization data 
pl.chosen_action = None # 2 #chosen action for the current trial

In [8]:
#Defining necessary function modules: 

#init_params.py: default neuronal values 
pl.celldefaults = par.helper_cellparams()
pl.popspecific = par.helper_popspecific()
pl.receptordefaults = par.helper_receptor()
pl.basestim = par.helper_basestim()
pl.dpmndefaults = par.helper_dpmn()
pl.d1defaults = par.helper_d1()
pl.d2defaults = par.helper_d2()
#pl.actionchannels = pl[par.helper_actionchannels]()

#popconstruct.py: default population parameters 
pl.popdata = pl[popconstruct.helper_popconstruct](pl.actionchannels, pl.popspecific, pl.celldefaults, pl.receptordefaults, pl.basestim, pl.dpmndefaults, pl.d1defaults, pl.d2defaults)
pl.pathways = pl[popconstruct.helper_poppathways](pl.popdata)

#popconstruct.py: to create connectivity grids
pl.connectivity_AMPA, pl.meaneff_AMPA, pl.plastic_AMPA = pl[popconstruct.helper_connectivity]('AMPA', pl.popdata, pl.pathways).shape(3)
pl.connectivity_GABA, pl.meaneff_GABA, pl.plastic_GABA = pl[popconstruct.helper_connectivity]('GABA', pl.popdata, pl.pathways).shape(3)
pl.connectivity_NMDA, pl.meaneff_NMDA, pl.plastic_NMDA = pl[popconstruct.helper_connectivity]('NMDA', pl.popdata, pl.pathways).shape(3)

In [9]:
#Adding codeblocks to the newtork pipeline: 
pl.add(codeblock_modifycelldefaults)
pl.add(codeblock_modifypopspecific)
pl.add(codeblock_modifyreceptordefaults)
pl.add(codeblock_modifybasestim)
pl.add(codeblock_modifydpmndefaults)
pl.add(codeblock_modifyd1defaults)
pl.add(codeblock_modifyd2defaults)
pl.add(codeblock_modifyactionchannels)
pl.add(codeblock_popconstruct)
pl.add(codeblock_poppathways)

### Q-values pipeline

In [10]:
#Creating a new pipeline for Q-values: 
q_val_pipe = cbgt.Pipeline()

In [11]:
#Defining necessary function modules: 

#qvalues.py
q_val_pipe.Q_support_params = q_val_pipe[qval.helper_init_Q_support_params]()
q_val_pipe.Q_df = q_val_pipe[qval.helper_init_Q_df](pl.actionchannels)

#rsg.reward_val = q_val_pipe[qval.get_reward_value](rsg.t1_epochs,rsg.t2_epochs,pl.chosen_action,pl.trial_num) 
#q_val_pipe.Q_support_params = q_val_pipe[qval.helper_update_Q_support_params](q_val_pipe.Q_support_params,rsg.reward_val,pl.chosen_action)
#(q_val_pipe.Q_df, q_val_pipe.Q_support_params, pl.dpmndefaults) = q_val_pipe[qval.helper_update_Q_df](q_val_pipe.Q_df,q_val_pipe.Q_support_params,pl.dpmndefaults,pl.trial_num).shape(3)

In [12]:
#Adding the q_val_pipe to the main network pipeline pl:
pl.add(q_val_pipe)

# Timestep loop

In [13]:
def mega_loop(slf):
    slf.AMPA_con,slf.AMPA_eff = CreateSynapses(slf.popdata,slf.connectivity_AMPA,slf.meaneff_AMPA,slf.plastic_AMPA)
    slf.GABA_con,slf.GABA_eff = CreateSynapses(slf.popdata,slf.connectivity_GABA,slf.meaneff_GABA,slf.plastic_GABA)
    slf.NMDA_con,slf.NMDA_eff = CreateSynapses(slf.popdata,slf.connectivity_NMDA,slf.meaneff_NMDA,slf.plastic_NMDA)

    popdata = slf.popdata
    actionchannels = slf.actionchannels
    agent = initializeAgent(popdata)
    slf.agent = agent

    agent.AMPA_con,agent.AMPA_eff = slf.AMPA_con,slf.AMPA_eff
    agent.GABA_con,agent.GABA_eff = slf.GABA_con,slf.GABA_eff
    agent.NMDA_con,agent.NMDA_eff = slf.NMDA_con,slf.NMDA_eff
    agent.LastConductanceNMDA = CreateAuxiliarySynapseData(popdata,slf.connectivity_NMDA)

    agent.FRs = agent.rollingbuffer.mean(1) / untrace(list(popdata['N'])) / agent.dt * 1000

    multitimestep_mutator(agent,popdata,1000)

    agent.phase = 0
    agent.phasetimer = 0
    agent.motor_queued = None
    agent.dpmn_queued = None
    agent.gain = np.ones(len(actionchannels))
    agent.extstim = np.zeros(len(actionchannels))
    agent.ramping_extstim = np.zeros(len(actionchannels))
    agent.in_popids = np.where(popdata['name'] == 'LIP')[0]
    agent.out_popids = np.where(popdata['name'] == 'Th')[0]

    presented_stimulus = 1
    slf.chosen_action = None

    while slf.trial_num < slf.n_trials:
        agent.extstim = agent.gain * presented_stimulus * 4.0  # TODO: make 3.0 a param
        agent.ramping_extstim = agent.ramping_extstim * 0.9 + agent.extstim * 0.1
        for action_idx in range(len(actionchannels)):
            popid = agent.in_popids[action_idx]
            agent.FreqExt_AMPA[popid] = np.ones(len(agent.FreqExt_AMPA[popid])) * agent.ramping_extstim[action_idx]
        multitimestep_mutator(agent,popdata,5)
        agent.phasetimer += 1 # 1 ms = 5 * dt
        #agent.FRs = np.stack((agent.FRs,agent.rollingbuffer.mean(1) / untrace(list(popdata['N'])) / agent.dt * 1000))

        if agent.phase == 0:
            gateFRs = agent.rollingbuffer[agent.out_popids].mean(1) / untrace(list(popdata['N'][agent.out_popids])) / agent.dt * 1000
            thresholds_crossed = np.where(gateFRs > 30)[0]
            if len(thresholds_crossed) > 0 or agent.phasetimer > 1000:
                agent.phase = 1
                agent.phasetimer = 0
                agent.gain = np.zeros(len(actionchannels))
                if len(thresholds_crossed) > 0:
                    agent.motor_queued = thresholds_crossed[0]
                    agent.gain[agent.motor_queued] = 0.5
                else:
                    agent.motor_queued = -1

        if agent.phase == 1:
            if agent.phasetimer > 100:
                agent.phase = 2
                agent.phasetimer = 0
                agent.gain = np.zeros(len(actionchannels))
                slf.chosen_action = untrace(actionchannels.iloc[agent.motor_queued,0])
                agent.motor_queued = None

        if agent.phase == 2:
            if agent.phasetimer > 100:
                slf.dpmndefaults['dpmn_DAp'] = 0
                slf.trial_num += 1
                agent.phase = 0
                agent.phasetimer = 0


        # environment

        if slf.chosen_action is not None:
            #slf.reward_val = qval.get_reward_value(slf.t1_epochs,slf.t2_epochs,slf.chosen_action,slf.trial_num)
            slf.reward_val = qval.get_reward_value(slf.t_epochs,slf.chosen_action,slf.trial_num)
            slf.Q_support_params = qval.helper_update_Q_support_params(slf.Q_support_params,slf.reward_val,slf.chosen_action)
            (slf.Q_df, slf.Q_support_params, slf.dpmndefaults) = qval.helper_update_Q_df(slf.Q_df,slf.Q_support_params,slf.dpmndefaults,slf.trial_num)
            slf.chosen_action = None

In [14]:
pl.add(mega_loop)

# Running the pipeline

### 1. Default values

In [15]:
#See init_params.py / generateepochs.py / popconstruct.py / qvalues.py
#to know the parameters contained in each of the following variables 
environment = {
    'params': None, #neuron parameters (init_params.py)
    'pops': None,   #population parameters (init_params.py)
    'receps' : None, #receptor parameters (init_params.py)
    'base' : None, #baseline stimulation parameters (init_params.py)
    'dpmns' : None, #dopamine related parameters (init_params.py)
    'd1' : None, #D1-MSNs population related parameters (init_params.py)
    'd2' : None, #D2-MSNs population related parameters (init_params.py)
    'channels' : None, #action channels related parameters (init_params.py)
    'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), #labels for the actions (init_params.py)
    #'actionchannels' : pd.DataFrame([["left"],["right"]], columns=['action']), #labels for the actions (init_params.py)
    'newpathways' : None, #connectivity parameters (popconstruct.py)
    'Q_support_params': None, #initialization of Q-values update (qvalues.py) 
    'Q_df':None, #initialized Q-values df (qvalues.py)  
    'n_trials': 5, #number of trials (generateepochs.py)
    'volatility': 2, #frequency of changepoints (generateepochs.py)
    'conflict': 0.7, #probability of the preferred choice (generateepochs.py)
    'reward_mu': 1, #mean for the magnitude of the reward (generateepochs.py)
    'reward_std': 0.1, #std for the magnitude of the reward (generateepochs.py)
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment)

(raylet) /home/bahuguna/anaconda3/envs/cbgt_env/lib/python3.6/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /home/bahuguna/anaconda3/envs/cbgt_env/lib/python3.6/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)
(raylet) /home/bahuguna/anaconda3/envs/cbgt_env/lib/python3.6/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `p

(pid=None)   action
(pid=None) 0   left
(pid=None) 1  right
(pid=None) define_epochs
(pid=None)   action
(pid=None) 0   left
(pid=None) 1  right
(pid=None) reward        left     right
(pid=None) 0  0.820854  0.000000
(pid=None) 1  0.000000  0.908504
(pid=None) 2  0.000000  0.985054
(pid=None) 3  1.008814  0.000000
(pid=None) 4  0.996019  0.000000
(pid=None)   action
(pid=None) 0   left
(pid=None) 1  right
(pid=None) define_epochs
(pid=None)   action
(pid=None) 0   left
(pid=None) 1  right
(pid=None) reward        left     right
(pid=None) 0  0.820854  0.000000
(pid=None) 1  0.000000  0.908504
(pid=None) 2  0.000000  0.985054
(pid=None) 3  1.008814  0.000000
(pid=None) 4  0.996019  0.000000
(pid=None) 2 4 0.161668`
(pid=None) 2 4 0.161668`
(pid=None) 3 5 0.161668`
(pid=None) 3 5 0.161668`
(pid=None) 10 6 1.0`
(pid=None) 10 8 1.0`
(pid=None) 10 6 1.0`
(pid=None) 10 8 1.0`
(pid=None) 10 10 0.4335`
(pid=None) 10 10 0.4335`
(pid=None) 10 11 0.4335`
(pid=None) 10 12 1.0`
(pid=None) 10 13 1.

RayTaskError(IndexError): [36mray::worker()[39m (pid=86144, ip=10.0.0.154)
  File "pandas/_libs/index.pyx", line 80, in pandas._libs.index.IndexEngine.get_value
  File "pandas/_libs/index.pyx", line 90, in pandas._libs.index.IndexEngine.get_value
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 2

During handling of the above exception, another exception occurred:

[36mray::worker()[39m (pid=86144, ip=10.0.0.154)
  File "python/ray/_raylet.pyx", line 490, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 497, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 501, in ray._raylet.execute_task
  File "/home/bahuguna/anaconda3/envs/cbgt_env/lib/python3.6/site-packages/ray/util/tracing/

In [ ]:
results.keys()

In [ ]:
results['popdata']

### 2. Modifying default values 

### 2.1 Modifying values 

In [ ]:
environment = {
    'params': pd.DataFrame([[8, 30]], columns=['N', 'C']),
    'pops' : {'FSI': {'Taum': [60]}}, 
    'receps' : pd.DataFrame([[100, 100]], columns=['Tau_AMPA', 'Tau_GABA']), 
    'base' : {'FSI': {'FreqExt_AMPA': [100]}}, 
    'dpmns' : pd.DataFrame([[400, 100]], columns=['dpmn_wmax', 'dpmn_tauDOP']),
    'd1' : pd.DataFrame([[3, 50]], columns=['dpmn_type', 'dpmn_c']),
    'd2' : pd.DataFrame([[3, 50]], columns=['dpmn_type', 'dpmn_c']),
    'channels' : None,
    'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), 
    'newpathways' : pd.DataFrame([['LIP', 'D1STR', 'AMPA', 'syn', 700, 0.027]], columns=['src', 'dest', 'receptor', 'type', 'con', 'eff']), 
    'Q_support_params': None,
    'Q_df':None,
    'n_trials': 5,
    'volatility': 2,
    'conflict': 0.7,
    'reward_mu': 1,
    'reward_std': 0.1,
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [ ]:
results['popdata']

### 2.2 Modifying a subset of values

In [ ]:
environment = {
    'params': pd.DataFrame([[4, 1]], columns=['N', 'C']), 
    'pops': None,
    'receps' : None, 
    'base' : None,
    'dpmns' : None,
    'd1' : None,
    'd2' : None,
    'channels' : None,
    'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), 
    'newpathways' : None, 
    'Q_support_params': None,
    'Q_df':None,
    'n_trials': 5,
    'volatility': 2,
    'conflict': 0.7,
    'reward_mu': 1,
    'reward_std': 0.1,
}

results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [ ]:
results['popdata']

### 3. Back to default values 

In [ ]:
#See init_params.py / generateepochs.py / popconstruct.py / qvalues.py
#to know the parameters contained in each of the following variables 
environment = {
    'params': None, #neuron parameters (init_params.py)
    'pops': None,   #population parameters (init_params.py)
    'receps' : None, #receptor parameters (init_params.py)
    'base' : None, #baseline stimulation parameters (init_params.py)
    'dpmns' : None, #dopamine related parameters (init_params.py)
    'd1' : None, #D1-MSNs population related parameters (init_params.py)
    'd2' : None, #D2-MSNs population related parameters (init_params.py)
    'channels' : None, #action channels related parameters (init_params.py)
    'actionchannels' : pd.DataFrame([[1],[2]], columns=['action']), #labels for the actions (init_params.py)
    'newpathways' : None, #connectivity parameters (popconstruct.py)
    'Q_support_params': None, #initialization of Q-values update (qvalues.py) 
    'Q_df':None, #initialized Q-values df (qvalues.py)  
    'n_trials': 5, #number of trials (generateepochs.py)
    'volatility': 2, #frequency of changepoints (generateepochs.py)
    'conflict': 0.7, #probability of the preferred choice (generateepochs.py)
    'reward_mu': 1, #mean for the magnitude of the reward (generateepochs.py)
    'reward_std': 0.1, #std for the magnitude of the reward (generateepochs.py)
}
results = cbgt.ExecutionManager(cores=7).run(pl,environment) 

In [ ]:
results['popdata']

In [ ]:
#?

In [ ]:
dev = cbgt.Pipeline()

In [ ]:
dev.add(codeblock_agent_phase0)

In [ ]:
output = cbgt.ExecutionManager(cores=7).run(dev,results) 